In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

In [ ]:
# Load dataset
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 18s 2us/step


In [7]:
print(f"Mnist Train Images shape: {mnist_train_images.shape}")
print(f"Mnist Test Images shape: {mnist_test_images.shape}")
print(f"Mnist Train  shape: {mnist_train_labels.shape}")
print(f"Mnist Train Images shape: {mnist_test_labels.shape}")

Mnist Train Images shape: (60000, 28, 28)
Mnist Test Images shape: (10000, 28, 28)
Mnist Train  shape: (60000,)
Mnist Train Images shape: (10000,)


In [8]:
# Normalize images
mnist_train_images = mnist_train_images.astype('float32') / 255.0
mnist_test_images = mnist_test_images.astype('float32') / 255.0

# Flatten images to vectors of size 784
mnist_train_images = mnist_train_images.reshape(-1, 28 * 28)
mnist_test_images = mnist_test_images.reshape(-1, 28 * 28)

In [9]:
# Hyperparameters
learning_rate = 0.1
num_epochs = 50
batch_size = 128
num_features = 28 * 28
num_classes = 10

In [10]:
# Shuffling and batching the data using tf.data API

train_dataset = tf.data.Dataset.from_tensor_slices((mnist_train_images, mnist_train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((mnist_test_images, mnist_test_labels))
test_dataset = test_dataset.batch(batch_size)


In [11]:
W = tf.Variable(tf.random.normal([num_features, num_classes], stddev=0.01))
b = tf.Variable(tf.zeros([num_classes]))

# Step 6: Defining logistic regression model and cost function
def logistic_regression(x):
    logits = tf.matmul(x, W) + b  # Linear computation
    return tf.nn.softmax(logits)  # Apply softmax to get probabilities

def cross_entropy_loss(y_true, y_pred):
    # Convert labels to one-hot vectors
    y_true_onehot = tf.one_hot(y_true, depth=num_classes)
    # Compute cross entropy loss
    loss = tf.reduce_mean(-tf.reduce_sum(y_true_onehot * tf.math.log(y_pred + 1e-10), axis=1))
    return loss

In [ ]:
# optimizer and accuracy metric
optimizer = tf.optimizers.SGD(learning_rate=learning_rate)


def compute_accuracy(y_true, y_pred):
    # Cast y_true to int32 to match tf.argmax's output type
    y_true = tf.cast(y_true, tf.int32)
    correct_preds = tf.equal(tf.argmax(y_pred, axis=1, output_type=tf.int32), y_true)
    return tf.reduce_mean(tf.cast(correct_preds, tf.float32))

In [13]:
# training

for epoch in range(num_epochs):
    epoch_loss = 0
    for step, (batch_x, batch_y) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            # Forward pass: compute predictions
            predictions = logistic_regression(batch_x)
            # Compute loss for the current batch
            loss = cross_entropy_loss(batch_y, predictions)
        # Step 8: Compute gradients
        gradients = tape.gradient(loss, [W, b])
        # Step 8: Update weights and biases using the optimizer
        optimizer.apply_gradients(zip(gradients, [W, b]))
        epoch_loss += loss.numpy()

    # Compute average loss for the epoch
    avg_loss = epoch_loss / (step + 1)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

Epoch 1/50, Loss: 0.5717
Epoch 2/50, Loss: 0.3737
Epoch 3/50, Loss: 0.3422
Epoch 4/50, Loss: 0.3257
Epoch 5/50, Loss: 0.3150
Epoch 6/50, Loss: 0.3073
Epoch 7/50, Loss: 0.3017
Epoch 8/50, Loss: 0.2970
Epoch 9/50, Loss: 0.2933
Epoch 10/50, Loss: 0.2901
Epoch 11/50, Loss: 0.2873
Epoch 12/50, Loss: 0.2848
Epoch 13/50, Loss: 0.2827
Epoch 14/50, Loss: 0.2807
Epoch 15/50, Loss: 0.2790
Epoch 16/50, Loss: 0.2775
Epoch 17/50, Loss: 0.2760
Epoch 18/50, Loss: 0.2747
Epoch 19/50, Loss: 0.2735
Epoch 20/50, Loss: 0.2723
Epoch 21/50, Loss: 0.2711
Epoch 22/50, Loss: 0.2702
Epoch 23/50, Loss: 0.2694
Epoch 24/50, Loss: 0.2685
Epoch 25/50, Loss: 0.2676
Epoch 26/50, Loss: 0.2670
Epoch 27/50, Loss: 0.2661
Epoch 28/50, Loss: 0.2654
Epoch 29/50, Loss: 0.2646
Epoch 30/50, Loss: 0.2640
Epoch 31/50, Loss: 0.2636
Epoch 32/50, Loss: 0.2629
Epoch 33/50, Loss: 0.2623
Epoch 34/50, Loss: 0.2617
Epoch 35/50, Loss: 0.2615
Epoch 36/50, Loss: 0.2608
Epoch 37/50, Loss: 0.2603
Epoch 38/50, Loss: 0.2598
Epoch 39/50, Loss: 0.

In [14]:
test_accuracy = 0
num_batches = 0
for batch_x, batch_y in test_dataset:
    predictions = logistic_regression(batch_x)
    batch_accuracy = compute_accuracy(batch_y, predictions)
    test_accuracy += batch_accuracy.numpy()
    num_batches += 1

test_accuracy /= num_batches
print(f"\nTest Accuracy: {test_accuracy * 100:.2f}%")


Test Accuracy: 92.52%
